In [33]:
import pandas as pd

data = pd.read_excel("data/LoadDocstore.xlsx", sheet_name=1)


In [34]:
# Cut off the first 3 rows and split into two dfs: balance + transactions
data = pd.read_excel("data/LoadDocstore.xlsx", sheet_name=1, skiprows=4, header=0)


In [35]:
# Split the data into two dataframes: balance + transactions based on first NaN row in Amount Col

# Find first blank row in Amount column
mask = pd.isna(data['Amount'])
tx = data.loc[:mask.idxmax()-1]
inv_tx = data.loc[mask.idxmax():]

# Find start of second table and remove preceding rows
mask = data['Date'].eq('Date')
inv_tx = inv_tx.loc[mask.idxmax():]

# Rename col headers for inv_tx table
inv_tx = inv_tx.rename(columns=inv_tx.iloc[0])
inv_tx = inv_tx.drop(inv_tx.index[0])

# Remove the total row
inv_tx = inv_tx.drop(inv_tx.index[-1])

In [36]:
columns = ['Date', 'Time', 'ShareID', 'Type', 'Shares', 'Price', 'Broker', 'Stamp', 'Noncash', 'Total', 'Note']


In [37]:
import_file_inv_tx = inv_tx.copy()
# Renames
import_file_inv_tx.rename(columns={'InvestmentName': 'SharepadID'}, inplace=True)
import_file_inv_tx.rename(columns={'Quantity': 'Shares'}, inplace=True)
import_file_inv_tx.rename(columns={'Cost': 'Total'}, inplace=True)

# Copy TransactionDetails for Type
import_file_inv_tx['Type'] = import_file_inv_tx['TransactionDetails']
import_file_inv_tx['Type'] = import_file_inv_tx['Type'].apply(lambda x: 'buy' if x.startswith('Bought') else 'sell' if x.startswith('Sold') else x)
import_file_inv_tx.rename(columns={'TransactionDetails': 'Note'}, inplace=True)


In [38]:
import_file_tx = tx.copy()

# Remove any rows starting with Bought or Sold or relate to Dividends
import_file_tx = import_file_tx[~import_file_tx['Details'].str.startswith(('Bought', 'Sold'))]
import_file_tx = import_file_tx[~import_file_tx['Details'].str.contains('DIV:')]

# Copy Details for Note
import_file_tx['Note'] = import_file_tx['Details']

# Change Details to Type
import_file_tx.loc[import_file_tx['Note'] == 'Regular Deposit', 'Details'] = 'credit'
import_file_tx.loc[import_file_tx['Note'].str.startswith('Account Fee'), 'Details'] = 'debit'
import_file_tx.loc[import_file_tx['Note'].str.startswith('Cash Account Interest'), 'Details'] = 'credit'
import_file_tx.loc[import_file_tx['Note'].str.contains('One-off withdrawal'), 'Details'] = 'debit'

# Remove any row starting with Div as SharePad can handle this

# Rename Cols
import_file_tx.rename(columns={'Details': 'Type'}, inplace=True)
import_file_tx.rename(columns={'Amount': 'Total'}, inplace=True)


# Remove cols
import_file_tx = import_file_tx[['Date', 'Type', 'Total']]



In [39]:
merged_df = pd.concat([import_file_tx, import_file_inv_tx], ignore_index=True)
merged_df['charges'] = 0
merged_df['stamp'] = 0
merged_df.to_csv('export/merged_df.csv', index=False)